In [ ]:
import matplotlib.pyplot as plt

import numpy as np

from keras.layers import Input, Conv2D, MaxPooling2D#, UpSampling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
from keras import backend as K

In [ ]:
K.set_image_data_format('channels_last')  # TF dimension ordering in this code

In [ ]:
#========================================================
#   Simple image plotting routine
#========================================================
def tvscale(img, plot_size=(6,6)):
    plt.rcParams['figure.figsize'] = plot_size
    imgplot = plt.imshow(img,interpolation="none",cmap='gray')
    plt.axis('off')
    plt.show()

In [ ]:
def get_model(n_pix,n_filters):
  input_img = Input(shape=(n_pix,n_pix,3))

#---- Bloque
  x1 = Conv2D(filters=n_filters, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv1')(input_img)
  x2 = MaxPooling2D((2, 2), padding='same', name='maxpool1')(x1) # 64,64,16

  x3 = Conv2D(filters=n_filters*2, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv2')(x2)
  x4 = MaxPooling2D((2, 2), padding='same', name='maxpool2')(x3) # 64,64,16

  x5 = Conv2D(filters=n_filters*4, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv3')(x4)
  x6 = MaxPooling2D((2, 2), padding='same', name='maxpool3')(x5) # 64,64,16

  x7 = Conv2D(filters=n_filters*8, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='conv4')(x6)
  x8 = MaxPooling2D((2, 2), padding='same', name='maxpool4')(x7) # 64,64,16

  flat = Flatten(name='flatten')(x8)
  x9 = Dense(32, activation='relu', name='dense1')(flat)
  x10 = Dropout(0.3, name='dropout')(x9)
  out = Dense(1, activation='sigmoid', name='out')(x10)

  _model = Model(inputs=input_img, outputs=out)
  _model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  return _model


In [ ]:
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
X1 = np.load('/content/drive/MyDrive/cosmo-late.npy')
X2 = np.load('/content/drive/MyDrive/cosmo-early.npy')

print(X1.shape)
print("type", type(X1[0,0,0,0]))

(301, 128, 128, 3)
type <class 'numpy.uint8'>


In [ ]:
#--- Concatenate early and late sets. Simple pre-processing
X = np.concatenate((X1,X2), axis=0)

#--- IMPORTANT!!! Normalize and convert to floating point!!!
X = X.astype(np.float32)/256.0

#--- Assign classes
Y = np.zeros((X.shape[0]))
Y[0:len(X1)] = 1 #--- Late is 0, early 1
print(Y.shape)


#--- Get random indexes
rand_ind = ( np.arange(0,X.shape[0]) )
np.random.shuffle(rand_ind)

#--- Shuffle data
X = X[rand_ind]
Y = Y[rand_ind]

(602,)


In [ ]:
#--- Define training and test datasets

X_train = X[50:]
Y_train = Y[50:]

X_test = X[0:50]
Y_test = Y[0:50]

In [ ]:
model = get_model(128,16)

In [ ]:
history = model.fit(X_train, Y_train, batch_size=32, epochs=29, shuffle=True)

Epoch 1/29
18/18 [==============================] - 13s 24ms/step - loss: 0.6208 - accuracy: 0.6721
Epoch 2/29
18/18 [==============================] - 0s 13ms/step - loss: 0.3194 - accuracy: 0.9058
Epoch 3/29
18/18 [==============================] - 0s 13ms/step - loss: 0.2902 - accuracy: 0.8931
Epoch 4/29
18/18 [==============================] - 0s 14ms/step - loss: 0.2316 - accuracy: 0.9312
Epoch 5/29
18/18 [==============================] - 0s 13ms/step - loss: 0.1864 - accuracy: 0.9312
Epoch 6/29
18/18 [==============================] - 0s 13ms/step - loss: 0.1494 - accuracy: 0.9493
Epoch 7/29
18/18 [==============================] - 0s 13ms/step - loss: 0.1537 - accuracy: 0.9384
Epoch 8/29
18/18 [==============================] - 0s 14ms/step - loss: 0.1385 - accuracy: 0.9583
Epoch 9/29
18/18 [==============================] - 0s 13ms/step - loss: 0.1281 - accuracy: 0.9601
Epoch 10/29
18/18 [==============================] - 0s 13ms/step - loss: 0.1032 - accuracy: 0.9674
Epoch 11